In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import LambdaCallback
from keras import backend as K
from keras import activations
import keras
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
from google.colab import drive
from scipy.stats import norm
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load arrays from Google Drive
X_train = np.load('/content/drive/My Drive/X_train_new.npy')
y_train = np.load('/content/drive/My Drive/y_train_new.npy')
X_test = np.load('/content/drive/My Drive/X_test_new.npy')
y_test = np.load('/content/drive/My Drive/y_test_new.npy')

In [4]:
percent_considered = 10. # the minimum difference between the % of neurons in range that we want to check

In [5]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(128, activation='relu', input_shape=(128,)))
model.add(Dense(11, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
ratio = 1
ds_size = len(X_train)  * ratio
batchsize_train = 128
dense_size = 128
n_batches = int(ds_size // batchsize_train)
n_epochs = 3
n_layers = 2

In [7]:
shapes = (n_batches, batchsize_train, dense_size)
layer_activations = []
for _ in range(n_layers):
  epochs = []
  for i in range(n_epochs):
    epochs.append(np.empty(shapes))
  layer_activations.append(epochs)
model_output = []
weights = []

class CustomCallback(Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        for i in range(n_layers):
          model_output.append(model.layers[i].output)
        # self.neuron_activations = []
        self.ok = False #used for testing

    def on_epoch_begin(self, epoch, logs=None):
      self.neuron_activations = []
      for i in range(n_layers):
        self.neuron_activations.append(layer_activations[i][epoch])
      if epoch == 0: ## replace 0 with the epoch you wanna test
        self.ok = True

    def on_batch_begin(self, batch, logs=None):
        get_activations = K.function([model.layers[0].input], model_output)
        if batch != (n_batches):  # we ignore the last, uncompleted batch so we can have a matrix
          get_act = get_activations([X_train[(batch) * batchsize_train: (batch + 1) * batchsize_train]])
          for i in range(n_layers):
            activations = get_act[i]
            self.neuron_activations[i][batch]= np.array(activations)

custom_callback = CustomCallback()

model.fit(X_train, y_train, epochs=n_epochs, batch_size=batchsize_train, callbacks=[custom_callback])


Epoch 1/3
  4/516 [..............................] - ETA: 11s - loss: 2.2283 - accuracy: 0.2578 

516/516 [==============================] - 11s 20ms/step - loss: 0.2884 - accuracy: 0.9184
Epoch 2/3
516/516 [==============================] - 8s 16ms/step - loss: 0.1152 - accuracy: 0.9657
Epoch 3/3
516/516 [==============================] - 7s 13ms/step - loss: 0.0793 - accuracy: 0.9767


Properly arrange list

In [8]:
neuron_activations = []
for j in range(n_layers):
  for i in range(n_epochs):
    layer_activations[j][i] = np.reshape(layer_activations[j][i], (n_batches * batchsize_train, dense_size ))
    layer_activations[j][i] = layer_activations[j][i].T.reshape((dense_size,-1))
  neuron_activations.append(np.hstack(layer_activations[j]))


Sort based on class


In [ ]:
activations_class = []
for layer_ in range(n_layers):
  layer = []
  for neuron_ in range(dense_size):
    neuron_class = []
    for class_ in range(11):
      neuron_class.append([])
    for image_ in range(n_epochs * n_batches * batchsize_train):
      label = np.argmax(y_train[image_ % (n_batches * batchsize_train)])
      neuron_class[label].append(neuron_activations[layer_][neuron_][image_])
    layer.append(neuron_class)
  activations_class.append(layer)

In [ ]:
(curr_layer, curr_neuron, curr_class) = 1, 2, 3
(curr_layer2, curr_neuron2, curr_class2) = 1, 2, 2

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].hist(activations_class[curr_layer][curr_neuron][curr_class], bins=20, color='skyblue', edgecolor='black')
axs[0].set_title(f'layer = {curr_layer} neuron = {curr_neuron} class = {curr_class}')
axs[0].set_xlabel('Value')
axs[0].set_ylabel('Frequency')

axs[1].hist(activations_class[curr_layer2][curr_neuron2][curr_class2], bins=20, color='salmon', edgecolor='black')
axs[1].set_title(f'layer = {curr_layer2} neuron = {curr_neuron2} class = {curr_class2}')
axs[1].set_xlabel('Value')
axs[1].set_ylabel('Frequency')

plt.tight_layout()

plt.show()

Standard deviation

In [ ]:
x_std = 1.5

In [ ]:
neurons_mstd = np.empty((n_layers, dense_size, 11,2))
for layer_ in range(n_layers):
  for neuron_ in range(dense_size):
    for class_ in range(11):
      mean = np.mean(activations_class[layer_][neuron_][class_])
      std = np.std(activations_class[layer_][neuron_][class_])
      neurons_mstd[layer_][neuron_][class_][0] = mean - x_std * std
      neurons_mstd[layer_][neuron_][class_][1] = mean + x_std * std
range_classes = neurons_mstd

Create random list to get out of range

In [ ]:
random_X_test = np.random.choice([-1., 1.], size=(11000, 784), p=[0.5, 0.5])
random_X_test += np.random.rand(11000, 784)  # Add random values between 0 and 1

Get activations from tests and random tests

In [ ]:
get_testing_a = K.function([model.layers[0].input],
                                  model_output)
get_label_pred = K.function([model.layers[0].input],
                                  [model.layers[-1].output])
layers_output_random = get_testing_a([random_X_test])
label_pred_random = get_label_pred([random_X_test])[0]

In [ ]:
max_indices = np.argmax(label_pred_random, axis=1) # having the right label eg 7 or 4
y_test_predict_random = np.zeros((max_indices.size, 11))
y_test_predict_random[np.arange(max_indices.size), max_indices] = 1 # one hot encoded

Find correlation

In [ ]:
def find_corr(array_v, array_bool):
  numeric_boolean_values = array_bool.astype(int)
  pearson_corr, pearson_p_value = pearsonr(array_v, numeric_boolean_values)
  spearman_corr, spearman_p_value = spearmanr(array_v, numeric_boolean_values)
  return pearson_corr, pearson_p_value, spearman_corr, spearman_p_value

Rearrange the list properly

In [ ]:
layers_output_random = np.array(layers_output_random)
layers_output_random = layers_output_random.transpose(0, 2, 1)

In [ ]:
n_images_random = len(random_X_test)
percentages_random = np.empty((n_images_random, 11))
for i in range(n_images_random):
  for class_ in range(11):
    count = 0
    for layer in range(n_layers):
      for neuron in range(dense_size):
        if range_classes[layer][neuron][class_][0] <= layers_output_random[layer][neuron][i] <= range_classes[layer][neuron][class_][1]:
          count +=1
    percentages_random[i][class_] = count / (n_layers * dense_size) * 100

In [ ]:
difference_list_rand = np.zeros(len(percentages_random), dtype=bool)
percentage_prediction_rand = np.zeros(len(percentages_random),dtype=bool)
interesting_rand = []
for percent_list_ in range(len(percentages_random)):
  percent_list = percentages_random[percent_list_]
  label_predict = np.argmax(percent_list)
  percentage_prediction_rand[percent_list_] = (label_predict == 10)
  check_difference = False
  for percent in percent_list:
    if abs(percent - percent_list[label_predict]) <= percent_considered and percent != percent_list[label_predict]:
      check_difference = True
      difference_list_rand[percent_list_] = True
      break
  if not check_difference :
    interesting_rand.append(percent_list)

Find Correlation and Accuracy for NonImage inputs

In [ ]:
n_wins_diff_rand = 0
for i in range(len(difference_list_rand)):
  if difference_list_rand[i] and percentage_prediction_rand[i]:
    n_wins_diff_rand += 1

In [ ]:
corr_p_rand, p_p_rand, corr_s_rand, s_p_rand = find_corr(difference_list_rand, percentage_prediction_rand)

In [ ]:
n_diff_rand = np.sum(difference_list_rand)
n_wins_rand = np.sum(percentage_prediction_rand)
print(f'Accuracy when there is a < 10% difference: {n_wins_diff_rand/n_diff_rand}')
print(f'Accuracy when there is a >= 10% difference: : {(n_wins_rand - n_wins_diff_rand)/(len(percentage_prediction_rand) - n_diff_rand)}')
print(f'Pearson Correlation: {corr_p_rand} p-value:{p_p_rand}')
print(f'Spearman Correlation: {corr_s_rand} p-value:{s_p_rand}')

Compute Correlation and Accuracy for Image input

In [ ]:
layers_output = get_testing_a([X_test])
label_pred = get_label_pred([X_test])[0]

In [ ]:
max_indices = np.argmax(label_pred, axis=1) # having the right label eg 7 or 4
y_test_predict = np.zeros((max_indices.size, 11))
y_test_predict[np.arange(max_indices.size), max_indices] = 1 # one hot encoded

In [ ]:
layers_output= np.array(layers_output)
layers_output = layers_output.transpose(0, 2, 1)

In [ ]:
n_images = len(y_test)
percentages = np.empty((n_images, 11))
for i in range(n_images):
  for class_ in range(11):
    count = 0
    for layer in range(n_layers):
      for neuron in range(dense_size):
        if range_classes[layer][neuron][class_][0] <= layers_output[layer][neuron][i] <= range_classes[layer][neuron][class_][1]:
          count +=1
    percentages[i][class_] = count / (n_layers * dense_size) * 100

Check which predictions are right

In [ ]:
a = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [ ]:
difference_list = np.zeros(len(percentages), dtype=bool)
percentage_prediction = np.zeros(len(percentages),dtype=bool)
for percent_list_ in range(len(percentages)):
  percent_list = percentages[percent_list_]
  label_predict = np.argmax(percent_list)
  percentage_prediction[percent_list_] = (label_predict == np.argmax(y_test[percent_list_]))
  check_difference = False
  for percent in percent_list:
    if abs(percent - percent_list[label_predict]) <= percent_considered and percent != percent_list[label_predict]:
      check_difference = True
      difference_list[percent_list_] = True
      break

In [ ]:
n_wins_diff = 0
for i in range(len(difference_list)):
  if difference_list[i] and percentage_prediction[i]:
    n_wins_diff += 1

In [ ]:
corr_p, p_p, corr_s, s_p = find_corr(difference_list, percentage_prediction)

In [ ]:
n_diff = np.sum(difference_list)
n_wins = np.sum(percentage_prediction)
print(f'Accuracy when there is a < 10% difference: {n_wins_diff/n_diff}')
print(f'Accuracy when there is a >= 10% difference: {(n_wins - n_wins_diff)/(len(percentage_prediction) - n_diff)}')
print(f'Pearson Correlation: {corr_p} p-value:{p_p}')
print(f'Spearman Correlation: {corr_s} p-value:{s_p}')

In [ ]:
av_perc_random = np.empty(len(percentages_random))
for i in range(len(av_perc_random)):
  av_perc_random[i] = np.sum(percentages_random[i])/11
print(f'average random percentage is {np.sum(av_perc_random)/len(av_perc_random)}')


In [ ]:
av_perc = np.empty(len(percentages))
for i in range(len(av_perc)):
  av_perc[i] = np.sum(percentages[i])/11
print(f'averagepercentage is {np.sum(av_perc)/len(av_perc)}')